In [4]:
import os
import glob
from tqdm import tqdm
import pydicom

import pandas as pd
import numpy as np

In [5]:
root = "/media/yesindeed/WD5T/data/Lung-PET-CT-Dx"

df_meta = pd.read_csv(os.path.join(root, "manifest", "metadata.csv"))

df_meta = df_meta.loc[df_meta["Modality"] == "CT"]
df_meta = df_meta.loc[df_meta["Number of Images"] >= 5]

df_meta = df_meta.reset_index(drop=True)

print(len(df_meta))
df_meta.head()

1135


,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,Chest,04-04-2007,5mm,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,64,33.75 MB,./Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-C...,2024-09-23T13:01:58.789
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.257508444832...,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,Chest,04-04-2007,5mm,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,64,33.75 MB,./Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-C...,2024-09-23T13:01:59.247
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.207154000096...,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0002,1.3.6.1.4.1.14519.5.2.1.6655.2359.104855313699...,ThoraxAThoraxRoutine Adult,04-25-2007,ThoraxRoutine 8.0.0 B40f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,30,15.82 MB,./Lung-PET-CT-Dx/Lung_Dx-A0002/04-25-2007-NA-T...,2024-09-23T13:02:00.699
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.241981550240...,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0002,1.3.6.1.4.1.14519.5.2.1.6655.2359.104855313699...,ThoraxAThoraxRoutine Adult,04-25-2007,ThoraxRoutine 8.0.0 B70f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,30,15.82 MB,./Lung-PET-CT-Dx/Lung_Dx-A0002/04-25-2007-NA-T...,2024-09-23T13:02:07.333
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.144797390935...,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0003,1.3.6.1.4.1.14519.5.2.1.6655.2359.179476503242...,ThoraxAThoraxRoutine Adult,07-07-2006,ThoraxRoutine 10.0 B40f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,27,14.24 MB,./Lung-PET-CT-Dx/Lung_Dx-A0003/07-07-2006-NA-T...,2024-09-23T13:02:09.076


In [6]:
dict_out = {"id": [], "exam_id": [], "path": [], "class": [], "exam_class": [], "patient": []}

class_mapping = {
    "A": "adenocarcinoma",
    "B": "small cell carcinoma",
    "E": "large cell carcinoma",
    "G": "squamous cell carcinoma",
}

for i in tqdm(range(len(df_meta))):
    series = df_meta.iloc[i]

    series_path = os.path.join(root, "manifest", series["File Location"])
    patient = series["Subject ID"].split("-")[-1]

    for dcm_file in glob.glob(os.path.join(series_path, "*.dcm")):
        dicom = pydicom.dcmread(dcm_file)
        dcm_id = dicom[0x0008, 0x0018].value

        dict_out["id"].append(dcm_id)
        dict_out["exam_id"].append(series["Series UID"])
        dict_out["path"].append("./" + os.path.join("manifest", series["File Location"], dcm_file.split("/")[-1]))

        assert os.path.exists(os.path.join(root, dict_out["path"][-1]))

        if os.path.exists(os.path.join(root, "Annotation", patient, f"{dcm_id}.xml")):
            cls = class_mapping[patient[0]]
        else:
            cls = "no findings"

        dict_out["class"].append(cls)
        dict_out["exam_class"].append(cls)

        dict_out["patient"].append(patient)

        break

df_out = pd.DataFrame.from_dict(dict_out)
df_out.head()

100%|██████████| 1135/1135 [00:05<00:00, 217.75it/s]


,id,exam_id,path,class,exam_class,patient
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.326712390675...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,./manifest/./Lung-PET-CT-Dx/Lung_Dx-A0001/04-0...,adenocarcinoma,adenocarcinoma,A0001
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.301905642463...,1.3.6.1.4.1.14519.5.2.1.6655.2359.257508444832...,./manifest/./Lung-PET-CT-Dx/Lung_Dx-A0001/04-0...,adenocarcinoma,adenocarcinoma,A0001
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.101887664051...,1.3.6.1.4.1.14519.5.2.1.6655.2359.207154000096...,./manifest/./Lung-PET-CT-Dx/Lung_Dx-A0002/04-2...,no findings,no findings,A0002
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.295499053390...,1.3.6.1.4.1.14519.5.2.1.6655.2359.241981550240...,./manifest/./Lung-PET-CT-Dx/Lung_Dx-A0002/04-2...,no findings,no findings,A0002
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.288787619309...,1.3.6.1.4.1.14519.5.2.1.6655.2359.144797390935...,./manifest/./Lung-PET-CT-Dx/Lung_Dx-A0003/07-0...,no findings,no findings,A0003


In [13]:
# df_out.to_csv(f"{root}/temp.csv", index=False)

In [14]:
df_out["class"].value_counts()

class
no findings    204836
Name: count, dtype: int64

In [7]:
df_out = pd.read_csv(f"{root}/temp.csv")

for i in tqdm(range(len(df_out))):
    series = df_out.iloc[i]

    dcm_id = series["id"]
    patient = series["patient"]

    if os.path.exists(os.path.join(root, "Annotation", patient, f"{dcm_id}.xml")):
        cls = class_mapping[patient[0]]
    else:
        cls = "no findings"

    df_out.iloc[i]["class"] = cls
    df_out.iloc[i]["exam_class"] = cls

df_out.to_csv(f"{root}/temp.csv", index=False)

  0%|          | 0/204836 [00:00<?, ?it/s]/tmp/ipykernel_1825193/973828781.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_out.iloc[i]["class"] = cls
/tmp/ipykernel_1825193/973828781.py:15: FutureWarning: ChainedAssignmentError: beha

In [10]:
df_out["class"].value_counts()

class
no findings                174593
adenocarcinoma              19927
squamous cell carcinoma      7081
small cell carcinoma         3034
large cell carcinoma          201
Name: count, dtype: int64

In [11]:
import numpy as np

df_out["dataset"] = "lung-pet-ct-dx"
df_out["class_num"] = 4
df_out["contains_negative"] = 1
df_out["modality"] = "ct"

df_out = df_out.sample(frac=1).reset_index(drop=True)

patients = df_out["patient"].unique()
patient_size = len(patients)

test_indices = np.random.choice(list(range(patient_size)), int(patient_size * 0.15), replace=False)

test_patients = patients[test_indices]

df_out["split"] = "train"

df_out.loc[df_out["patient"].isin(test_patients), "split"] = "test"

In [12]:
df_out = df_out.drop("patient", axis=1)

df_out.to_csv(f"{root}/metadata.csv", index=False)